In [1]:
import pandas as pd
import os
from bidi.algorithm import get_display
import re

In [2]:
import smtplib
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

In [2]:
send_email()

successfully sent the mail


In [1]:
def send_email(user='dsakaidf@gmail.com', pwd='d54k4idf!', recipient='shkasta@post.bgu.ac.il',
               subject='finish expirement', body='my password is d54k4idf!'):
    import smtplib

    gmail_user = user
    gmail_pwd = pwd
    FROM = user
    TO = recipient if type(recipient) is list else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    try:
        # SMTP_SSL Example
        server_ssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server_ssl.ehlo()  # optional, called by login()
        server_ssl.login(gmail_user, gmail_pwd)
        # ssl server doesn't support or need tls, so don't call server_ssl.starttls()
        server_ssl.sendmail(FROM, TO, message)
        # server_ssl.quit()
        server_ssl.close()
        print('successfully sent the mail')
    except:
        print("failed to send mail")


In [2]:
import stanza

In [3]:
he_nlp = stanza.Pipeline('he', processors='tokenize,mwt', verbose=False, use_gpu=False)
#he_nlp = stanza.Pipeline('he', processors='tokenize,lemma,pos', verbose=False, use_gpu=False)

In [4]:
stanza.download('he', verbose=True)

2020-04-28 23:41:17 INFO: Downloading default packages for language: he (Hebrew)...
2020-04-28 23:41:17 INFO: File exists: C:\Users\Michael\stanza_resources\he\default.zip.
2020-04-28 23:41:20 INFO: Finished downloading models and saved to C:\Users\Michael\stanza_resources.


In [5]:
threads_df = None

In [6]:
type_file = 'post'

In [7]:
field_name = 'message'

In [8]:
file_name = os.listdir('data/')
for filename in file_name:
    if type_file in filename:
        temp_df = pd.read_csv(f'data/{filename}')
        if threads_df is None:
            threads_df = temp_df
        else:
            threads_df = threads_df.append(temp_df)

In [9]:
dataset = threads_df

In [10]:
dataset.head()

,Unnamed: 0,thread_id,post_id,user_name,date,message,cite1,cite2,cite3,cite4
0,0,20126916,post_202834374,רומא,21-02-2020 20:54,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,login.php?do=lostpw,login.php?do=lostpw,login.php?do=lostpw,NaN
1,1,20126916,post_202834407,רומא,21-02-2020 20:54,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,login.php?do=lostpw,login.php?do=lostpw,login.php?do=lostpw,NaN
2,2,20126916,post_202834485,רומא,21-02-2020 20:54,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,login.php?do=lostpw,login.php?do=lostpw,login.php?do=lostpw,NaN
3,3,20126916,post_202834494,רומא,21-02-2020 20:54,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,login.php?do=lostpw,login.php?do=lostpw,login.php?do=lostpw,NaN
4,4,20126916,post_202834534,רומא,21-02-2020 20:54,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,login.php?do=lostpw,login.php?do=lostpw,login.php?do=lostpw,NaN


In [11]:
#Fetch wordcount for each abstract
dataset['word_count'] = dataset[field_name].apply(lambda x: len(str(x).split(" ")))
dataset[[field_name,'word_count']].head()

,message,word_count
0,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,35
1,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,35
2,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,35
3,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,35
4,\nיש כאלה בכלל?\n \n \n \n טיס עד לגיבוש \nשח...,35


In [12]:
dataset.word_count.describe()

count    334819.000000
mean        224.657269
std         177.378942
min           1.000000
25%         110.000000
50%         186.000000
75%         290.000000
max        5242.000000
Name: word_count, dtype: float64

In [13]:
freq = pd.Series(' '.join(dataset[field_name]).split()).value_counts()[:20]
freq

לא     1935550
זה     1462097
אני    1037347
את      939085
אתה     778459
על      706923
מה      686145
אם      668095
לי      657111
אבל     585003
יש      560402
לך      558872
של      548140
עם      464913
או      434318
גם      405743
כל      381187
אז      369157
הוא     299469
כי      297944
dtype: int64

In [14]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('stopwords.txt', 'r', encoding='utf8') as f:
    text = f.read()
stop_words = set(word for word in text.split('\r\n'))

In [ ]:
corpus = []
count = 0
for i in dataset[field_name]:
    text = []
    try:        
        he_doc = he_nlp(re.sub('- FXP', '', i))
    except:
        count += 1
        continue
    for i, sent in enumerate(he_doc.sentences):
        for word in reversed(sent.words):
            word_text = word.text
            if not '_' in word_text and not word_text in stop_words and len(word_text) > 1:
                try:
                    text.append(get_display(word_text))
                except:
                    print(word.text)
    if len(text) > 0:
        corpus.append(' '.join(text))
print(f'number remove setence {count}')

..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
🤭🙅*♂
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
🤨זה
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
אחשלייייייייייייי⁦✌️⁩
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
🤨שקט
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
👨*🦰
🦹*♂
👨*🦰
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
יורד😄לי🥯על
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
👶🏻⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
⁦❤️⁩
🥰🥰
🥰🥰


In [ ]:
corpus[:10]

In [ ]:
dataset.shape

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline
wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stop_words,
                          max_words=100,
                          max_font_size=50, 
                          random_state=42,
                          font_path='C:/WINDOWS/Fonts/Arial.ttf'
                         ).generate(' '.join(corpus))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=1200)

In [ ]:
new_corpus = list()
for sentence in corpus:
    new_corpus.append(' '.join([word for word in sentence.split() if not word in stop_words]))

In [ ]:
corpus = new_corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(new_corpus)

In [ ]:
list(cv.vocabulary_.keys())[:10]

In [ ]:
!pip install seaborn
import seaborn

In [ ]:
#Most frequently occuring words
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]
#Convert most freq words to dataframe for plotting bar plot
top_words = get_top_n_words(corpus, n=20)
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]
#Barplot of most freq words
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
g = sns.barplot(x="Word", y="Freq", data=top_df)
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
#Most frequently occuring Bi-grams
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2),  
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]
top2_words = get_top_n2_words(corpus, n=20)
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
print(top2_df)
#Barplot of most freq Bi-grams
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Bi-gram", y="Freq", data=top2_df)
h.set_xticklabels(h.get_xticklabels(), rotation=45)

In [ ]:
#Most frequently occuring Tri-grams
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3), 
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]
top3_words = get_top_n3_words(corpus, n=20)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
print(top3_df)
#Barplot of most freq Tri-grams
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
j=sns.barplot(x="Tri-gram", y="Freq", data=top3_df)
j.set_xticklabels(j.get_xticklabels(), rotation=45)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)
# get feature names
feature_names=cv.get_feature_names()
 
# fetch document for which keywords needs to be extracted
doc=corpus[532]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

In [ ]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)
 
# now print the results
print("\nAbstract:")
print(doc)
print("\nKeywords:")
for k in keywords:
    print(k,keywords[k])